In [1]:
import os
import sys
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as f
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

import cv2
from PIL import Image as im
from sklearn.metrics import jaccard_score

import collections
from typing import DefaultDict, Tuple, List, Dict
from functools import partial

sys.path.append('early-stopping-pytorch')
from pytorchtools import EarlyStopping

In [ ]:
# Retrieve unique ID used to name a model when saving files related to it.
# The current integer is the previously trained model. Training will increment the value.
COUNTER_FILENAME = os.path.expanduser("model_counter.txt")
global MODEL_ID


def update_model_id():
    global MODEL_ID
    with open(COUNTER_FILENAME, "w") as f:
        count = int(MODEL_ID)
        count += 1
        count = str(count)
        f.write(count)
        MODEL_ID = count

        
def retrieve_model_id():
    global MODEL_ID
    try:
        with open(COUNTER_FILENAME, 'r') as f:
            count = f.read()
            MODEL_ID = count
    except FileNotFoundError:
        print('New counter file.')
        with open(COUNTER_FILENAME, 'w') as f:
            count = '0'
            f.write(count)
            MODEL_ID = count

            
retrieve_model_id()

# Adjust printing view dimensions
np.set_printoptions(threshold=sys.maxsize, linewidth=300)
torch.set_printoptions(threshold=sys.maxsize, linewidth=300, profile='full')

In [12]:
class VAE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        
        self.encoder = nn.Sequentual(
                            # Initial tensor is batch_sizex1x30x30
                            nn.Conv2d(
                                in_channels=1, 
                                out_channels=32,
                                kernel_size=3, 
                                stride=2, 
                                padding=1),
                            nn.BatchNorm2d(32),
                            nn.ReLU(),
                            # tensor is batch_sizex32x15x15 since:
                            #   ((30 + (2 * 1) - (1 * (3 - 1)) - 1) / 2) + 1
                            # = ((30 + 2 - 2 - 1) / 2) + 1
                            # = (29 / 2) + 1 = 14 + 1 = 15
                            nn.Conv2d(
                                in_channels=32, 
                                out_channels=64,
                                kernel_size=3, 
                                stride=2, 
                                padding=1),
                            nn.BatchNorm2d(64),
                            nn.ReLU(),
                            # tensor is batch_sizex64x8x8 since:
                            # ((15 + (2 * 1) - (1 * (3 - 1)) - 1) / 2) + 1
                            # = ((15 + 2 - 2 - 1) / 2) + 1
                            # = (14 / 2) + 1 = 8
                            nn.Conv2d(
                                in_channels=64, 
                                out_channels=900,
                                kernel_size=3, 
                                stride=2, 
                                padding=1),
                            nn.BatchNorm2d(900),
                            nn.ReLU())
                            # tensor is batch_sizex900x4x4 since:
                            # ((8 + (2 * 1) - (1 * (3 - 1)) - 1) / 2) + 1
                            # = ((8 + 2 - 2 - 1) / 2) + 1
                            # = (7 / 2) + 1 = 4
        
        # 900 * 4 * 4 = 14400
        self.mean = nn.Linear(14400, 4)
        self.log_var = nn.Linear(14400, 4)
        
        self.decoder_input = nn.Linear(4, 14400)
        
        self.decoder = nn.Sequential(
                            nn.ConvTranspose2d(
                                in_channels=900,
                                out_channels=64,
                                kernel_size=3,
                                stride = 2,
                                padding=1,
                                output_padding=1),
                            nn.BatchNorm2d(64),
                            nn.ReLU(),
                            nn.ConvTranspose2d(
                                in_channels=64,
                                out_channels=32,
                                kernel_size=3,
                                stride = 2,
                                padding=1,
                                output_padding=1),
                            nn.BatchNorm2d(32),
                            nn.ReLU())
        
        self.final_layer = nn.Sequential(
                                nn.ConvTranspose2d(
                                    in_channels=32,
                                    out_channels=32,
                                    kernel_size=3,
                                    stride=2,
                                    padding=1,
                                    output_padding=1),
                                nn.BatchNorm2d(16),
                                nn.LeakyReLU(),
                                nn.Conv2d(
                                    in_channels=32, 
                                    out_channels=1,
                                    kernel_size=3, 
                                    padding=1),
                                nn.Sigmoid())
        
        
    def encode(self, data):
        result = self.encoder(data)
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mean = self.mean(result)
        log_var = self.log_var(result)

        return [mean, log_var]

    
    def decode(self, z):
        result = self.decoder_input(z)
        result = result.view(-1, 900, 2, 2)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result

    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    
    def forward(self, data, **kwargs):
        mean, log_var = self.encode(data)
        z = self.reparameterize(mean, log_var)
        return [self.decode(z), data, mean, log_var]
    
    
    def loss_function(self, *args, **kwargs):
        recons = args[0]
        data = args[1]
        mean = args[2]
        log_var = args[3]

        kld_weight = kwargs['beta'] # Account for the minibatch samples from the dataset
        recons_loss = nn.BCEloss(recons, data)

        kld = torch.mean(-0.5 * torch.sum(1 + log_var - mean ** 2 - log_var.exp(), dim = 1), dim = 0)

        loss = recons_loss + kld_weight * kld
        return {'loss': loss, 'Reconstruction_Loss':recons_loss.detach(), 'KLD':-kld_loss.detach()}

    
    def sample(self, num_samples, current_device, **kwargs):
        z = torch.randn(num_samples,
                        self.latent_dim)
        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    
    def generate(self, x, **kwargs):
        return self.forward(x)[0]

In [4]:
# Load the dataset
# 10,000 samples, 30x30 matrices
data_count = 10000
data = np.ndarray(shape=(data_count,30,30))
n_features = data.shape[1] * data.shape[2]


for i in range(data_count):
    path = f'data/jet_matrices/sample{i+1}.dat'
    sample = np.loadtxt(path, unpack = False)
    data[i] = sample

print("Done loading data.")

Done loading data.


In [5]:
# Load parameters corresponding to the 4 variables input into 
# the Helmholtz Resonator function, where output is each sample in dataset.
params = np.ndarray(shape=(data_count,4))

path = r'data/param_lhs.dat'
with open(path) as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        if i >= params.shape[0]:
            break
        param = np.fromstring(line, dtype=float, sep=',')
        params[i] = param

print("Done loading parameters.")

Done loading parameters.


In [6]:
# Convert from numpy array to Pytorch tensor
X = torch.from_numpy(data)
# Convert all scalars to floats
X = X.float()
print(X.shape)

X_with_params = []
for i in range(data_count):
    pair = [X[i], params[i], i]
    X_with_params.append(pair)

torch.Size([10000, 30, 30])


In [8]:
# Hyperparameters

batch_size = 32
# 70/15/15 split
train_size = int(0.7 * len(X))
val_test_size = len(X) - train_size
test_size = val_test_size // 2
    
val_size = val_test_size - test_size

print(train_size)
print(val_test_size)
print(test_size)
print(val_size)

7000
3000
1500
1500


In [9]:
# Initate data loaders

train, val = torch.utils.data.random_split(X_with_params, [train_size, val_test_size], generator=torch.Generator().manual_seed(5))
val, test = torch.utils.data.random_split(val, [val_size, test_size], generator=torch.Generator().manual_seed(5))

train_loader = torch.utils.data.DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    val, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    test, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True
)

# Same as test_loader but with stochastic batch size
test_loader_stoch = torch.utils.data.DataLoader(
    test, batch_size=1, shuffle=False, num_workers=0, pin_memory=True
)

# Use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
def train_validate(model, epochs, lr, is_early_stopping=False, patience=None, beta=None, rho=None):
    # Define Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Binary Cross Entropy Loss
    criterion = nn.BCELoss()
    
    # Reset model state
    def weights_init(m):
        if type(m) == nn.Sequential:
            children = dict(m.named_children())
            # Use Kaiming if ReLU, Xavier if Sigmoid.
            if type(children['1']) == nn.ReLU:
                nn.init.kaiming_uniform_(children['0'].weight.data, nonlinearity='relu')
            else:
                nn.init.xavier_uniform_(children['0'].weight.data)
            
    model.apply(weights_init)
    
    # Toggle Early Stopping (if using).
    if is_early_stopping:
        early_stopping = EarlyStopping(patience=patience, verbose=True)
        print("Using Early Stopping")

    print("Training...")
    for epoch in range(epochs):
        
        #############################    
        #          TRAINING         #
        #############################
        
        loss = 0
        # Prepare model for training
        model.train()
        train_losses = []
        for i, batch in enumerate(train_loader, 0):
            # remove params, keep data
            batch = batch[0]
            # reshape mini-batch data from [batch_size, 30, 30] to [batch_size, 900]
            # load it to the active device
            batch = batch.view(-1, n_features).to(device)

            # reset the gradients back to zero
            # PyTorch accumulates gradients on subsequent backward passes
            optimizer.zero_grad()

            # compute reconstructions
            # also retrieve bottleneck weights for computing sparsity penalty
            decoded = model(batch)

            # Exception handler for when BCE loss has values outside range [0.0, 1.0]
            try:
                # Compute training reconstruction loss
                train_loss = criterion(decoded, batch)
            except RuntimeError:
                print('Runtime Error during loss calculation. BCE loss has values outside range [0.0, 1.0]')
                for k, sample in enumerate(decoded):
                    print(k)
                    print(sample)
                    
            train_loss.backward()
            optimizer.step()

In [10]:
lr = 1e-3
epochs = 200

# Boolean for whether to use Early Stopping
is_early_stopping = True
# early stopping patience; how long to wait after last time validation loss improved.
patience = 10

torch.manual_seed(5)

basic_model = VAE(input_shape=n_features,
                    n_units=n_units,
                    latent_units=latent_units
                   ).to(device)

# vae_trained = train_validate(model=basic_model,
#                             epochs=epochs,
#                             lr=lr,
#                             is_early_stopping=is_early_stopping, 
#                             is_pca=is_pca,
#                             patience=patience)

NameError: name 'n_units' is not defined